In [1]:
# imports
import pandas as pd
import numpy as np
import re
import nltk
from tqdm.notebook import tqdm
from IPython.core.interactiveshell import InteractiveShell
from nltk.corpus import stopwords

In [19]:
# import configurations
InteractiveShell.ast_node_interactivity = "all"
tqdm.pandas()
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chayisara/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [67]:
english_stop_words = stopwords.words('english')
add_stopwords = ['bitcoin', 'btc', 'crypto', 'cryptocurrency', 'cryptocurrencies',
                'ethereum', 'eth', 'price', 'prices', 'money', 'dollars']
remove_stopwords = ['up','down']
# if 'sell' in english_stop_words:
#     print('True')
# else:
#     print('False')
english_stop_words = english_stop_words + add_stopwords
for i in remove_stopwords:
    if i in english_stop_words:
        english_stop_words.remove(i)
# if 'down' in english_stop_words:
#     print('True')
# else:
#     print('False')


In [55]:
# read file
df = pd.read_pickle("../data/interim/tweets_verified_2020-2021.pkl")
df.head()
df.shape
df.shape
df[(df['id'] == 1212277383287328769) | (df['id'] == 1212277146401402880)]

,created_at,id,conversation_id,user_id,username,name,tweet,language,mentions,urls,...,replies_count,retweets_count,likes_count,hashtags,cashtags,link,quote_url,video,thumbnail,reply_to
0,2020-01-01 00:08:28-05:00,1212239143687741440,1212239143687741440,20646945,dumbfoundead,dumbfoundead.eth,Bitcoin the worst decision i made this decade,en,[],[],...,9,15,409,[],[],https://twitter.com/dumbfoundead/status/1212239143687741440,<NA>,False,<NA>,[]
1,2020-01-01 00:54:35-05:00,1212250748815208448,1212250748815208448,636023721,neuroecology,Adam J Calhoun,"If you ask someone what they'd do if they went back ten years, they'd all use information to beat the stock market, buy bitcoin etc. What do you know now that would give you a head start on science? Like, CRISPR I guess?",en,[],[],...,6,0,4,[],[],https://twitter.com/neuroecology/status/1212250748815208448,<NA>,False,<NA>,[]
2,2020-01-01 02:00:25-05:00,1212267316789952512,1212267316789952512,631810714,cnbctv18news,CNBC-TV18,"#bitcoin rally begun in 2013 and it reached a peak of $20,000 in 2017 https://t.co/W3ghBhpfMX",en,[],['https://www.cnbctv18.com/market/currency/bitcoin-rises-9000000-in-a-decade-outperforms-all-indices-asset-class-4974831.htm'],...,0,0,4,['bitcoin'],[],https://twitter.com/CNBCTV18News/status/1212267316789952512,<NA>,False,<NA>,[]
3,2020-01-01 02:18:48-05:00,1212271940502638593,1212271940502638593,14654494,excellion,Samson Mow,"Wishing all #Bitcoin Ultra Enthusiasts a very Happy New Year! There’s so much to look forward to in 2020: improved privacy on both Layer 1 &amp; 2, more #LiquidNetwork growth, new products &amp; tech from @Blockstream, @InfiniteFleet from @Pixelmatic, &amp; 2 @magicalcrypto conferences! 🥂🎉 https://t.co/8pNY0QgcMz",en,"[{'screen_name': 'blockstream', 'name': 'blockstream', 'id': '2307692107'}, {'screen_name': 'infinitefleet', 'name': 'infinite fleet', 'id': '2376662816'}, {'screen_name': 'pixelmatic', 'name': 'pixelmatic', 'id': '15088549'}, {'screen_name': 'magicalcrypto', 'name': 'magical crypto friends', 'id': '935007382827962369'}]",[],...,10,17,158,"['bitcoin', 'liquidnetwork']",[],https://twitter.com/Excellion/status/1212271940502638593,<NA>,True,https://pbs.twimg.com/media/ENLa5toU0AAvhC1.jpg,[]
4,2020-01-01 02:39:29-05:00,1212277146401402880,1212277146401402880,1066972567943053312,hindustantimes,Hindustan Times,"Breaking down Bitcoin’s 9,000,000% rise in last decade that left the skeptics aghast https://t.co/9478QJTIsh",en,[],['http://www.hindustantimes.com/tech/breaking-down-bitcoin-s-9-000-000-rise-in-last-decade-that-left-the-skeptics-aghast/story-QehVZ3EeeEvlGBUyHt1x5O.html'],...,0,0,0,[],[],https://twitter.com/HindustanTimes/status/1212277146401402880,<NA>,False,<NA>,[]


(312521, 21)

(312521, 21)

,created_at,id,conversation_id,user_id,username,name,tweet,language,mentions,urls,...,replies_count,retweets_count,likes_count,hashtags,cashtags,link,quote_url,video,thumbnail,reply_to
4,2020-01-01 02:39:29-05:00,1212277146401402880,1212277146401402880,1066972567943053312,hindustantimes,Hindustan Times,"Breaking down Bitcoin’s 9,000,000% rise in last decade that left the skeptics aghast https://t.co/9478QJTIsh",en,[],['http://www.hindustantimes.com/tech/breaking-down-bitcoin-s-9-000-000-rise-in-last-decade-that-left-the-skeptics-aghast/story-QehVZ3EeeEvlGBUyHt1x5O.html'],...,0,0,0,[],[],https://twitter.com/HindustanTimes/status/1212277146401402880,<NA>,False,<NA>,[]
5,2020-01-01 02:40:25-05:00,1212277383287328769,1212277383287328769,3305925577,httech,Hindustan Times Tech,"Breaking down Bitcoin’s 9,000,000% rise in last decade that left the skeptics aghast @htTweets https://t.co/mogsEsmkSz",en,"[{'screen_name': 'httweets', 'name': 'hindustan times', 'id': '36327407'}]",['https://www.hindustantimes.com/tech/breaking-down-bitcoin-s-9-000-000-rise-in-last-decade-that-left-the-skeptics-aghast/story-QehVZ3EeeEvlGBUyHt1x5O.html'],...,0,0,0,[],[],https://twitter.com/HTTech/status/1212277383287328769,<NA>,False,<NA>,[]


In [56]:
def repl(matchObj):
    char = matchObj.group(1)
    return "%s%s" % (char, char)

def remove_stop_words(corpus):
    removed_stop_words = []
    for review in corpus:
        removed_stop_words.append(
            ' '.join([word for word in review.split() 
                      if word not in english_stop_words])
        )
    return removed_stop_words

def clean_tweet_1(tweet):
    # remove URL "https://t.co/"
    new_url = re.sub(r"https://t.co/[A-Za-z0-9]+", " ", tweet)
    
    # remove mention
    new_mention = re.sub(r"@[A-Za-z0-9!#%&*;_\$\.]+", " ", new_url)
    
    # remove # ,turning hashtags to the typical words.
    new_symbol = re.sub(r"\W+", " ", new_mention)
    
    # remove number
    new_number = re.sub(r"[0-9]", "", new_symbol)
    
    return new_number

def clean_tweet_2(tweet):
    # reduce character sequences >3 to 3
    new_sequence = re.sub(re.compile(r"(\w)\1+"), repl, tweet)
    
    # replace 2 or more spaces with a single space.
    new_space = re.sub(r"\s+", " ", new_sequence)
    
    # remove spaces front and back
    new_space_end = re.sub(r"^\s+|\s$", "", new_space)
    
    return new_space_end

def clean_tweet_money(tweet):

    new_money = re.sub(r"([0-9]+) ?(k)", r"\1000", tweet)
    remove_money = re.sub(r"([0-9]+)(,?)([0-9]+)(,?)([0-9]+?)", r"\1\3\5", new_money)
    all_money = re.findall(r"([0-9]+)", remove_money)
    
    return all_money



In [57]:
# tweet = '3K'
# tweets = '9,000'
# new_money = re.sub(r"([0-9]+) ?(k)", r"\1000", tweet)
# remove_money = re.sub(r"([0-9]+)(,?)([0-9]+)(,?)([0-9]+?)", r"\1\3\5", tweets)
# new_money
# remove_money

In [58]:
# language EN only
df = df[df['language'] == 'en']
df['tweet_clean'] = pd.DataFrame(df['tweet'].str.lower())

In [59]:
# apply money extract
all_money = df['tweet_clean'].apply(clean_tweet_money)
all_money

0         []                        
1         []                        
2         [2013, 20000, 2017, 3]    
3         [2020, 1, 2, 2, 8, 0]     
4         [9000000, 9478]           
               ...                  
312516    []                        
312517    []                        
312518    []                        
312519    [2022, 76, 4, 1, 76, 4, 1]
312520    []                        
Name: tweet_clean, Length: 312204, dtype: object

In [60]:
# apply cleaning
df['tweet_clean'] = df['tweet_clean'].progress_apply(clean_tweet_1)
df = df.drop_duplicates(subset='tweet_clean', keep=False)
df['tweet_clean'] = remove_stop_words(df['tweet_clean'])
df['tweet_clean'] = df['tweet_clean'].progress_apply(clean_tweet_2)
df['tweet_clean']

  0%|          | 0/312204 [00:00<?, ?it/s]

  0%|          | 0/253613 [00:00<?, ?it/s]

0         worst decision made decade                                                                                                                                              
1         ask someone went back ten years use information beat stock market buy etc know would give head start science like crispr guess                                          
2         rally begun reached peak                                                                                                                                                
3         wishing ultra enthusiasts happy new year much look forward improved privacy layer amp liquidnetwork growth new products amp tech amp conferences                        
6         predictions crash sub usd hardware wallets hacked tron xrp become important satoshi identity revealed disappointing trump stack sats twitter lightning glorious adoption
                                                                                            ...          

In [62]:
df.head()
pd.set_option("display.max_colwidth", -1)
df[['tweet_clean', 'tweet']][300:400]

,created_at,id,conversation_id,user_id,username,name,tweet,language,mentions,urls,...,retweets_count,likes_count,hashtags,cashtags,link,quote_url,video,thumbnail,reply_to,tweet_clean
0,2020-01-01 00:08:28-05:00,1212239143687741440,1212239143687741440,20646945,dumbfoundead,dumbfoundead.eth,Bitcoin the worst decision i made this decade,en,[],[],...,15,409,[],[],https://twitter.com/dumbfoundead/status/1212239143687741440,<NA>,False,<NA>,[],worst decision made decade
1,2020-01-01 00:54:35-05:00,1212250748815208448,1212250748815208448,636023721,neuroecology,Adam J Calhoun,"If you ask someone what they'd do if they went back ten years, they'd all use information to beat the stock market, buy bitcoin etc. What do you know now that would give you a head start on science? Like, CRISPR I guess?",en,[],[],...,0,4,[],[],https://twitter.com/neuroecology/status/1212250748815208448,<NA>,False,<NA>,[],ask someone went back ten years use information beat stock market buy etc know would give head start science like crispr guess
2,2020-01-01 02:00:25-05:00,1212267316789952512,1212267316789952512,631810714,cnbctv18news,CNBC-TV18,"#bitcoin rally begun in 2013 and it reached a peak of $20,000 in 2017 https://t.co/W3ghBhpfMX",en,[],['https://www.cnbctv18.com/market/currency/bitcoin-rises-9000000-in-a-decade-outperforms-all-indices-asset-class-4974831.htm'],...,0,4,['bitcoin'],[],https://twitter.com/CNBCTV18News/status/1212267316789952512,<NA>,False,<NA>,[],rally begun reached peak
3,2020-01-01 02:18:48-05:00,1212271940502638593,1212271940502638593,14654494,excellion,Samson Mow,"Wishing all #Bitcoin Ultra Enthusiasts a very Happy New Year! There’s so much to look forward to in 2020: improved privacy on both Layer 1 &amp; 2, more #LiquidNetwork growth, new products &amp; tech from @Blockstream, @InfiniteFleet from @Pixelmatic, &amp; 2 @magicalcrypto conferences! 🥂🎉 https://t.co/8pNY0QgcMz",en,"[{'screen_name': 'blockstream', 'name': 'blockstream', 'id': '2307692107'}, {'screen_name': 'infinitefleet', 'name': 'infinite fleet', 'id': '2376662816'}, {'screen_name': 'pixelmatic', 'name': 'pixelmatic', 'id': '15088549'}, {'screen_name': 'magicalcrypto', 'name': 'magical crypto friends', 'id': '935007382827962369'}]",[],...,17,158,"['bitcoin', 'liquidnetwork']",[],https://twitter.com/Excellion/status/1212271940502638593,<NA>,True,https://pbs.twimg.com/media/ENLa5toU0AAvhC1.jpg,[],wishing ultra enthusiasts happy new year much look forward improved privacy layer amp liquidnetwork growth new products amp tech amp conferences
6,2020-01-01 02:54:22-05:00,1212280892103770114,1212280892103770114,918711,ourielohayon,OOeb3,2020 predictions Bitcoin will crash to sub-1000 USD all hardware wallets will be hacked Tron/XRP will become the most important cryptocurrency Satoshi identity will be revealed and will be disappointing Trump will stack sats on twitter Lightning will have glorious adoption,en,[],[],...,12,33,[],[],https://twitter.com/OurielOhayon/status/1212280892103770114,<NA>,False,<NA>,[],predictions crash sub usd hardware wallets hacked tron xrp become important satoshi identity revealed disappointing trump stack sats twitter lightning glorious adoption


/var/folders/09/339ff8256958pb0n3nhwztww0000gn/T/ipykernel_63343/331848320.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("display.max_colwidth", -1)


,tweet_clean,tweet
345,scammer asking people interested investments,Here’s the scammer asking people if they are interested in Bitcoin investments. https://t.co/ZjPPmtNKnp
346,started proof keys celebration monetary sovereignty said rights never flex muscles get weak keys,"“I started Proof of Keys as a celebration of our monetary sovereignty,” @TraceMayer said. “If you have rights but never flex them, your muscles will get weak. Not your keys, not your bitcoin. https://t.co/Upidj5dHeK"
347,date elusive satoshi nakamoto mined first block known genesis block started blockchain included message times jan chancellor brink second bailout banks,"On this date in 2009, the elusive Satoshi Nakamoto mined the first #bitcoin block. Known as the Genesis Block, it started the bitcoin #blockchain and included the message ""The Times 03/Jan/2009 Chancellor on brink of second bailout for banks."" https://t.co/voRoKMWWDU https://t.co/fQSI8UZPiu"
348,icymi featured shared vision behind river goal becoming financial institution tools offer performance tracking read,"#ICYMI: @Leishman was featured on @BitcoinMagazine and shared the vision behind River, our goal of becoming a #Bitcoin financial institution, and some of the tools we offer such as performance tracking. Read more: 👇👇👇 https://t.co/siJ6D8mqrH https://t.co/wztmfuXCcN"
349,calling maxi,@sakak_musdom Are you calling me a Bitcoin Maxi? 😀
...,...,...
445,pac discord referral competition follow link enter pacglobal discord referral competition st place month takes home pac global coins pac blockchain masternodes giveaways,"🚨PAC Discord Referral Competition 🚨 Follow the @Medium link below to enter the #PACGlobal Discord Referral Competition! 🥇1st place for the month takes home 250,000 PAC Global coins❗️ #PAC #Crypto #blockchain #bitcoin #Masternodes #giveaways https://t.co/n8hvWZ9RKG https://t.co/HqhOsbvl6q"
446,week completes muir glacier hard fork carnegie mellon sets up defi research launches euro pairs google play removes ban metamask still dead onwards,This week in crypto: 1. Ethereum completes Muir Glacier hard fork 2. Carnegie Mellon sets up DeFi research 3. @binance launches euro pairs 4. Google Play removes ban on Metamask 5. Bitcoin still not dead :) Onwards!
448,individual reason like said different people different places different reasons,"@MrHodl @BrianLockhart @bitcoin_burner @_k3m__ @ElkimXOC @BtcpayServer @NodedPodcast @stephanlivera @TFTC21 There is no individual reason. Like you said, different people in different places have different reasons."
449,utility real determinant value scaling original bsv protocol massively scales also platform within law outside law crushed,"@rogerkver utility is the only real determinant of value and this is all about scaling and only original #Bitcoin #BSV protocol massively scales. Also if a platform is not within the law, its outside the law and will be crushed."


In [68]:
# export as pickle
df = df.astype({'tweet':'string'}).reset_index(drop=True)
df.to_pickle("../data/cleaned/tweets_verified_2020-2021_cleaned.pkl")

In [28]:
# try reading pickle
# df_test = pd.read_pickle("../data/cleaned/tweets_verified_2020-2021_cleaned.pkl")

# display(df_test.dtypes)
# display(df_test)

In [77]:
# export as pickle
all_money.to_pickle("../data/cleaned/all_money.pkl")